# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

United Kingdom capital city's **London** is comparable to **New York City (NYC)** with the population of totaling around **8.9 million** compared to NYC’s **8.4 million**. However, London area covers around 607 square miles which is about twice as big as NYC’s 302.6 square mile area. (https://sarahfunky.com/travel-tips/2020/06/16/considering-a-move-the-differences-between-living-in-nyc-vs-london/)

There are **32 boroughs in London** and **5 boroughs in New York**. (https://en.wikipedia.org/wiki/London_boroughs) (https://www.nycgo.com/neighborhoods-boroughs/about-nyc-five-boroughs) 

Among them, **Manhattan of New York City** is one of the best areas of New York depends on this reference (https://www.insightguides.com/destinations/north-america/united-states/new-york-state/new-york-city/city-areas). On the other hand, **Westminster of London** is the best-performing borough in 2019 depends on this reference (https://www.cbreresidential.com/uk/en-GB/news/news-hot-100-2019-londons-best-performing-boroughs-ranked).

There is an article which mentioned about **“Considering a Move? The Differences Between Living in NYC Vs London”** (https://sarahfunky.com/travel-tips/2020/06/16/considering-a-move-the-differences-between-living-in-nyc-vs-london/).
Both cities are very diverse and attractive as well as well-known for many people.

The Business problem is **“which is the top 5 most common venues in Manhattan and Westminster?”**, **“What is the top 10 venues for each neighborhood in Manhattan and Westminster?”**.

**"Solving this problem may help the person who wants to choose to live between Manhattan and Westminster in the future by considering the venues around the neighborhoods".**

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* numbers of venues which is in common or top 10 venues for each neighborhood in Manhattan and Westminster

I decided to use to define our neighborhoods

Following data sources will be needed to extract/generate the required information:
* number of venues and their type and location in every neighborhood will be obtained using **Foursquare API**


**Steps Briefly Explanation**

After I **download** the data, I will check the neighborhoods data and transform it into a **pandas dataframe**. 
I will use **geopy library** to get the latitude and longitude values of **New York city and London**. 
Then creating a **map of New York City and London with neighborhoods** superimposed on top. 
As soon as I get this data, I will use **Folium** to make simplify the map and **segment and cluster only the neighborhoods in Manhattan of NY city and Westminster of London**. 
For this step, I will use my **Foursquare Credentials and start utilizing the Foursquare API to explore the neighborhoods and segment them**. 
Using **GET request URL** to get the unique categories from the returned venues. 
After all, I will **analyze each neighborhood and cluster neighborhoods**. 
Finally, I will **examine clusters**. 
Then I will provide for the **final data** and draw my conclusion based on the results.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
%matplotlib inline
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [14]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [15]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [16]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [17]:
CLIENT_ID = '4CICHCTBYVS5XHRUFYZJJ3VL5FWGWRN01T4QYMCJX3ZUGLNB'
CLIENT_SECRET = '3UYNS2L31JVNSPHDBYPYC5LAGZWJE5031JGSDBKQ5SYDORK4'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4CICHCTBYVS5XHRUFYZJJ3VL5FWGWRN01T4QYMCJX3ZUGLNB
CLIENT_SECRET:3UYNS2L31JVNSPHDBYPYC5LAGZWJE5031JGSDBKQ5SYDORK4


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [20]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3001, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Rite Aid,40.875467,-73.908906,Pharmacy
4,Marble Hill,40.876551,-73.91066,Subway,40.874667,-73.909586,Sandwich Place


In [21]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,63,63,63,63,63,63
Carnegie Hill,71,71,71,71,71,71
Central Harlem,40,40,40,40,40,40
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,85,85,85,85,85,85
Clinton,100,100,100,100,100,100
East Harlem,39,39,39,39,39,39
East Village,100,100,100,100,100,100


In [22]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 324 uniques categories.


In [23]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Gym,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,S

In [24]:
manhattan_onehot.shape

(3001, 325)

In [25]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Gym,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,S

In [26]:
manhattan_grouped.shape

(40, 325)

**Print each neighborhood along with the top 5 most common venues**

In [27]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.10
1          Hotel  0.06
2  Memorial Site  0.05
3            Gym  0.05
4    Coffee Shop  0.05


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.07
1  Pizza Place  0.06
2    Wine Shop  0.06
3          Gym  0.04
4         Café  0.04


----Central Harlem----
                  venue  freq
0    African Restaurant  0.08
1   Fried Chicken Joint  0.05
2                   Bar  0.05
3   American Restaurant  0.05
4  Gym / Fitness Center  0.05


----Chelsea----
                 venue  freq
0          Art Gallery  0.09
1               Bakery  0.05
2          Coffee Shop  0.05
3    French Restaurant  0.03
4  American Restaurant  0.03


----Chinatown----
                venue  freq
0              Bakery  0.08
1  Chinese Restaurant  0.06
2        Cocktail Bar  0.05
3      Ice Cream Shop  0.03
4  Salon / Barbershop  0.03


----Civic Center----
                  venue  freq
0           Coffee Shop  0.07
1                 

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Create the new dataframe and display the top 10 venues for each neighborhood**

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Memorial Site,Gym,Coffee Shop,Boat or Ferry,Sandwich Place,Chinese Restaurant,Gourmet Shop,Pizza Place
1,Carnegie Hill,Coffee Shop,Pizza Place,Wine Shop,Gym,Café,Cosmetics Shop,Bookstore,Japanese Restaurant,Gym / Fitness Center,Bakery
2,Central Harlem,African Restaurant,Fried Chicken Joint,Bar,American Restaurant,Gym / Fitness Center,Sandwich Place,Seafood Restaurant,French Restaurant,Juice Bar,Gym
3,Chelsea,Art Gallery,Bakery,Coffee Shop,French Restaurant,American Restaurant,Ice Cream Shop,Italian Restaurant,Hotel,Cycle Studio,Cupcake Shop
4,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,Ice Cream Shop,Salon / Barbershop,American Restaurant,Dessert Shop,Optical Shop,Spa,Noodle House


In [30]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 1, 1, 1, 1, 0, 1, 1], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Yoga Studio,Clothing Store,Tennis Stadium,Pizza Place,Storage Facility,Miscellaneous Shop,Ice Cream Shop,Coffee Shop,Bank
1,Manhattan,Chinatown,40.715618,-73.994279,1,Bakery,Chinese Restaurant,Cocktail Bar,Ice Cream Shop,Salon / Barbershop,American Restaurant,Dessert Shop,Optical Shop,Spa,Noodle House
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Pizza Place,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Spanish Restaurant,Bank,Bakery,Sandwich Place,Park
3,Manhattan,Inwood,40.867684,-73.921210,2,Café,Mexican Restaurant,Lounge,Deli / Bodega,Pizza Place,Bank,Bakery,Park,Restaurant,Wine Bar
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Sandwich Place,Mexican Restaurant,Café,Juice Bar,Bakery,Latin American Restaurant,Indian Restaurant,Cocktail Bar,Caribbean Restaurant


In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examine Clusters**

Cluster 1

In [33]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Pizza Place,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Spanish Restaurant,Bank,Bakery,Sandwich Place,Park
4,Hamilton Heights,Pizza Place,Sandwich Place,Mexican Restaurant,Café,Juice Bar,Bakery,Latin American Restaurant,Indian Restaurant,Cocktail Bar,Caribbean Restaurant
7,East Harlem,Bakery,Mexican Restaurant,Pizza Place,Bank,Deli / Bodega,Sandwich Place,Steakhouse,New American Restaurant,Thai Restaurant,French Restaurant
20,Lower East Side,Chinese Restaurant,Bakery,Art Gallery,Sandwich Place,Pizza Place,Pharmacy,Café,Latin American Restaurant,Cocktail Bar,Dance Studio


Cluster 2

In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,Ice Cream Shop,Salon / Barbershop,American Restaurant,Dessert Shop,Optical Shop,Spa,Noodle House
8,Upper East Side,Italian Restaurant,Exhibit,Bakery,Spa,Juice Bar,Wine Shop,Hotel,Coffee Shop,Yoga Studio,American Restaurant
9,Yorkville,Italian Restaurant,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gym,Deli / Bodega,Bar,Diner,Wine Shop,Spa
10,Lenox Hill,Italian Restaurant,Café,Pizza Place,Sushi Restaurant,Bank,Gym,Cocktail Bar,Coffee Shop,Deli / Bodega,Gym / Fitness Center
12,Upper West Side,Italian Restaurant,Bakery,Coffee Shop,Wine Bar,Café,American Restaurant,Cosmetics Shop,Vegetarian / Vegan Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant
13,Lincoln Square,Concert Hall,Café,Plaza,Theater,Performing Arts Venue,Bakery,Gym / Fitness Center,Indie Movie Theater,Clothing Store,Hotel
14,Clinton,Coffee Shop,Theater,Sandwich Place,American Restaurant,Italian Restaurant,Gym / Fitness Center,Gym,Spa,Steakhouse,Cocktail Bar
15,Midtown,Hotel,Clothing Store,Coffee Shop,Theater,Spa,Bakery,Sushi Restaurant,Bookstore,Food Truck,Cuban Restaurant
16,Murray Hill,Coffee Shop,Japanese Restaurant,Hotel,Gym / Fitness Center,Taco Place,American Restaurant,Pizza Place,Sandwich Place,Spa,Sushi Restaurant
17,Chelsea,Art Gallery,Bakery,Coffee Shop,French Restaurant,American Restaurant,Ice Cream Shop,Italian Restaurant,Hotel,Cycle Studio,Cupcake Shop


Cluster 3

In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Café,Mexican Restaurant,Lounge,Deli / Bodega,Pizza Place,Bank,Bakery,Park,Restaurant,Wine Bar
5,Manhattanville,Bank,Deli / Bodega,Chinese Restaurant,Coffee Shop,Italian Restaurant,Mexican Restaurant,Shipping Store,Indian Restaurant,Ice Cream Shop,Climbing Gym
6,Central Harlem,African Restaurant,Fried Chicken Joint,Bar,American Restaurant,Gym / Fitness Center,Sandwich Place,Seafood Restaurant,French Restaurant,Juice Bar,Gym
11,Roosevelt Island,Deli / Bodega,Park,Coffee Shop,Restaurant,Café,Outdoors & Recreation,Bus Line,Gym,Gym / Fitness Center,Bubble Tea Shop
26,Morningside Heights,Bookstore,Café,Sandwich Place,Deli / Bodega,American Restaurant,Park,Coffee Shop,Burger Joint,Bubble Tea Shop,Pharmacy
28,Battery Park City,Park,Hotel,Memorial Site,Gym,Coffee Shop,Boat or Ferry,Sandwich Place,Chinese Restaurant,Gourmet Shop,Pizza Place
35,Turtle Bay,Italian Restaurant,Park,Steakhouse,Deli / Bodega,Sushi Restaurant,Coffee Shop,French Restaurant,Seafood Restaurant,Turkish Restaurant,Sandwich Place
36,Tudor City,Park,Café,Deli / Bodega,Mexican Restaurant,Bank,Greek Restaurant,Gym / Fitness Center,Sushi Restaurant,Pizza Place,Thai Restaurant
39,Hudson Yards,Gym / Fitness Center,American Restaurant,Park,Restaurant,Gym,Coffee Shop,Hotel,Burger Joint,Italian Restaurant,Bus Line


Cluster 4

In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Boat or Ferry,Park,Yoga Studio,Pet Service,Bar,Coffee Shop,Bistro,Cocktail Bar,Farmers Market,Heliport


Cluster 5

In [37]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Yoga Studio,Clothing Store,Tennis Stadium,Pizza Place,Storage Facility,Miscellaneous Shop,Ice Cream Shop,Coffee Shop,Bank


In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
%matplotlib inline

In [39]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_areas_of_London')
# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

**Webscrapping**

In [40]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of areas of London - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b45381e4-23f4-4174-a507-e50bbf404172","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_areas_of_London","wgTitle":"List of areas of London","wgCurRevisionId":1018033845,"wgRevisionId":1018033845,"wgArticleId":11915713,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Use dmy dates from August 2015","Use 

In [41]:
table = soup.find("table",class_='wikitable sortable')
tbody = table.find('tbody')
trs = tbody.find_all('tr')

In [42]:
data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 0):
            data_table.append(td.text.strip())
        count=count+1;
df2 = pd.DataFrame(data_table)
df = pd.DataFrame(data_table,columns=['Location'])

In [43]:
df.head()

,Location
0,Abbey Wood
1,Acton
2,Addington
3,Addiscombe
4,Albany Park


In [44]:
data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 1):
            data_table.append(td.text.strip())
        count=count+1;
df[['Borough']] = pd.DataFrame(data_table)

In [45]:
data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 2):
            data_table.append(td.text.strip())
        count=count+1;
df[['Post town']] = pd.DataFrame(data_table)

In [46]:
data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 3):
            data_table.append(td.text.strip())
        count=count+1;
df[['Postcode district']] = pd.DataFrame(data_table)

In [47]:
data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 4):
            data_table.append(td.text.strip())
        count=count+1;
df[['Dial code']] = pd.DataFrame(data_table)

In [48]:
data_table=[]
count=0;
for tr in trs:
    tds = tr.find_all('td')
    for td in tds:
        if((count)%6 == 5):
            data_table.append(td.text.strip())
        count=count+1;
df[['OS grid ref']] = pd.DataFrame(data_table)

In [49]:
df.head()

,Location,Borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [50]:
df['Borough'] = df['Borough'].replace(regex={r'\[[1-9][0-9]]':''})
df.head()

,Location,Borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [51]:
df_proc = df[['Postcode district', 'Borough', 'Location']]
df_proc.rename(columns={"Postcode district": "Postcode", 
                        "Location":"Neighbourhood"},
               inplace=True)
df_proc.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postcode,Borough,Neighbourhood
0,SE2,"Bexley, Greenwich [7]",Abbey Wood
1,"W3, W4","Ealing, Hammersmith and Fulham[8]",Acton
2,CR0,Croydon[8],Addington
3,CR0,Croydon[8],Addiscombe
4,"DA5, DA14",Bexley,Albany Park


In [52]:
df_proc2 = df_proc.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_proc2.head()

,Postcode,Borough,Neighbourhood
0,BR1,Bromley,"Bromley, Plaistow, Sundridge, Widmore (also Wi..."
1,BR1,Lewisham,Downham
2,BR2,Bromley,"Hayes, Keston, Leaves Green, Southborough"
3,BR3,Bromley,"Bickley, Bromley Common, Eden Park, Elmers End"
4,"BR3, SE20",Bromley,Beckenham


In [53]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 98 kB 10.8 MB/s eta 0:00:01


In [54]:
import time

In [57]:
df_proc2.shape

(322, 3)

In [58]:
# Geocoder starts here
# Defining a function to use --> get_latlng()'''
def get_latlng(arcgis_geocoder):
    
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Geocoder ends here

In [64]:
start = time.time()
postal_codes = df_proc2['Postcode']    
coordinates = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]
end = time.time()
print("Time of execution: ", end - start, "seconds")

Time of execution:  262.5527286529541 seconds


In [65]:
df_se_loc = df_proc2
# The obtained coordinates (latitude and longitude) are joined with the dataframe as shown
df_se_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_se_loc['Latitude'] = df_se_coordinates['Latitude']
df_se_loc['Longitude'] = df_se_coordinates['Longitude']
df_se_loc.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,BR1,Bromley,"Bromley, Plaistow, Sundridge, Widmore (also Wi...",51.416710,0.009042
1,BR1,Lewisham,Downham,51.416710,0.009042
2,BR2,Bromley,"Hayes, Keston, Leaves Green, Southborough",51.507408,-0.127699
3,BR3,Bromley,"Bickley, Bromley Common, Eden Park, Elmers End",51.415095,-0.035403
4,"BR3, SE20",Bromley,Beckenham,51.482490,0.119194


In [68]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_se_loc['Borough'].unique()),
        df_se_loc.shape[0]
    )
)

The dataframe has 63 boroughs and 322 neighborhoods.


In [69]:
address = 'London'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [75]:
df_se_loc.drop('Postcode', inplace=True, axis=1)

In [76]:
df_se_loc.head(5)

,Borough,Neighbourhood,Latitude,Longitude
0,Bromley,"Bromley, Plaistow, Sundridge, Widmore (also Wi...",51.416710,0.009042
1,Lewisham,Downham,51.416710,0.009042
2,Bromley,"Hayes, Keston, Leaves Green, Southborough",51.507408,-0.127699
3,Bromley,"Bickley, Bromley Common, Eden Park, Elmers End",51.415095,-0.035403
4,Bromley,Beckenham,51.482490,0.119194


In [79]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_se_loc['Latitude'], df_se_loc['Longitude'], df_se_loc['Borough'], df_se_loc['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [80]:
westminster_data = df_se_loc[df_se_loc['Borough'] == 'Westminster'].reset_index(drop=True)
westminster_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Westminster,"Lisson Grove, St John's Wood",51.537125,-0.181958
1,Westminster,"Belgravia, Knightsbridge, Millbank, Pimlico, S...",51.504456,-0.140785
2,Westminster,"Chinatown, Marylebone (also St Marylebone), Ma...",51.507408,-0.127699
3,Westminster,"Bayswater, Paddington",51.515390,-0.192100
4,Westminster,Maida Vale,51.507408,-0.127699


In [81]:
address = 'Westminster'

geolocator = Nominatim(user_agent="westminster_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Westminster are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Westminster are 51.5004439, -0.1265398.


In [82]:
# create map of Westminster using latitude and longitude values
map_westminster = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(westminster_data['Latitude'], westminster_data['Longitude'], westminster_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_westminster)  
    
map_westminster

In [83]:
CLIENT_ID = '4CICHCTBYVS5XHRUFYZJJ3VL5FWGWRN01T4QYMCJX3ZUGLNB'
CLIENT_SECRET = '3UYNS2L31JVNSPHDBYPYC5LAGZWJE5031JGSDBKQ5SYDORK4'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4CICHCTBYVS5XHRUFYZJJ3VL5FWGWRN01T4QYMCJX3ZUGLNB
CLIENT_SECRET:3UYNS2L31JVNSPHDBYPYC5LAGZWJE5031JGSDBKQ5SYDORK4


In [84]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [85]:
westminster_venues = getNearbyVenues(names=westminster_data['Neighbourhood'],
                                   latitudes=westminster_data['Latitude'],
                                   longitudes=westminster_data['Longitude']
                                  )

Lisson Grove, St John's Wood
Belgravia, Knightsbridge, Millbank, Pimlico, St James's, Westminster
Chinatown, Marylebone (also St Marylebone), Mayfair, Soho
Bayswater, Paddington
Maida Vale
Little Venice
Aldwych, Charing Cross, Covent Garden


In [86]:
print(westminster_venues.shape)
westminster_venues.head()

(592, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lisson Grove, St John's Wood",51.537125,-0.181958,Chayada Thai,51.537248,-0.185291,Thai Restaurant
1,"Lisson Grove, St John's Wood",51.537125,-0.181958,Yasmeen Restaurant,51.535251,-0.181345,Middle Eastern Restaurant
2,"Lisson Grove, St John's Wood",51.537125,-0.181958,The Clifton,51.535858,-0.185155,Pub
3,"Lisson Grove, St John's Wood",51.537125,-0.181958,Bruno's Deli,51.537184,-0.183752,Deli / Bodega
4,"Lisson Grove, St John's Wood",51.537125,-0.181958,Belgrave Restaurant,51.537085,-0.183836,Persian Restaurant


check how many venues were returned for each neighborhood

In [87]:
westminster_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Aldwych, Charing Cross, Covent Garden",100,100,100,100,100,100
"Bayswater, Paddington",100,100,100,100,100,100
"Belgravia, Knightsbridge, Millbank, Pimlico, St James's, Westminster",79,79,79,79,79,79
"Chinatown, Marylebone (also St Marylebone), Mayfair, Soho",100,100,100,100,100,100
"Lisson Grove, St John's Wood",13,13,13,13,13,13
Little Venice,100,100,100,100,100,100
Maida Vale,100,100,100,100,100,100


how many unique categories can be curated from all the returned venues

In [88]:
print('There are {} uniques categories.'.format(len(westminster_venues['Venue Category'].unique())))

There are 126 uniques categories.


Analyze Each Neighborhood

In [89]:
# one hot encoding
westminster_onehot = pd.get_dummies(westminster_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
westminster_onehot['Neighbourhood'] = westminster_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [westminster_onehot.columns[-1]] + list(westminster_onehot.columns[:-1])
westminster_onehot = westminster_onehot[fixed_columns]

westminster_onehot.head()

,Neighbourhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Bakery,Bar,Beer Bar,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Building,Burger Joint,Burrito Place,Café,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,English Restaurant,Event Space,Falafel Restaurant,Fish & Chips Shop,Food Stand,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gelato Shop,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Multiplex,Museum,Nightclub,Noodle House,North Indian Restaurant,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Palace,Park,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Road,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Spa,Spanish Restaurant,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Tour Provider,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Lisson Grove, St John's Wood",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,"Lisson Grove, St John's Wood",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Lisson Grove, St John's Wood",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Lisson Grove, St John's Wood",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Lisson Grove, St John's Wood",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [90]:
westminster_onehot.shape

(592, 127)

In [91]:
westminster_grouped = westminster_onehot.groupby('Neighbourhood').mean().reset_index()
westminster_grouped

,Neighbourhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Bakery,Bar,Beer Bar,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Building,Burger Joint,Burrito Place,Café,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,English Restaurant,Event Space,Falafel Restaurant,Fish & Chips Shop,Food Stand,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gelato Shop,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Multiplex,Museum,Nightclub,Noodle House,North Indian Restaurant,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Palace,Park,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Road,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Spa,Spanish Restaurant,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Tour Provider,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Aldwych, Charing Cross, Covent Garden",0.00,0.01,0.01,0.010000,0.01,0.000000,0.00,0.00,0.02,0.04,0.01,0.000000,0.01,0.01,0.000000,0.00,0.000000,0.01,0.02,0.02,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.100000,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.010000,0.01,0.00,0.00,0.000000,0.00,0.040000,0.00,0.000000,0.000000,0.00,0.00,0.01,0.00,0.000000,0.00,0.030000,0.00,0.01,0.010000,0.03,0.050000,0.010000,0.00,0.000000,0.00,0.00,0.030000,0.00,0.030000,0.000000,0.01,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.01,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.020000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.120000,0.01,0.070000,0.000000,0.09,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.020000,0.010000,0.000000,0.03,0.00,0.00,0.000000,0.000000,0.00,0.01,0.010000,0.000000
1,"Bayswater, Paddington",0.01,0.00,0.00,0.010000,0.00,0.000000,0.01,0.01,0.01,0.01,0.00,0.010000,0.00,0.00,0.000000,0.01,0.000000,0.00,0.00,0.00,0.070000,0.000000,0.03,0.02,0.00,0.000000,0.010000,0.040000,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.00,0.000000,0.010000,0.00,0.01,0.01,0.000000,0.00,0.010000,0.00,0.000000,0.030000,0.01,0.00,0.01,0.01,0.000000,0.03,0.020000,0.01,0.03,0.000000,0.00,0.040000,0.000000,0.00,0.040000,0.00,0.00,0.020000,0.00,0.000000,0.010000,0.00,0.000000,0.02,0.00,0.000000,0.01,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.01,0.00,0.00,0.000000,0.01,0.00,0.01,0.00,0.00,0.000000,0.000000,0.040000,0.01,0.01,0.040000,0.000000,0.000000,0.01,0.080000,0.00,0.020000,0.000000,0.02,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.02,0.00,0.01,0.00,0.02,0.010000,0.000000,0.020000,0.00,0.00,0.00,0.000000,0.010000,0.02,0.01,0.000000,0.000000
2,"Belgravia, Knightsbridge, Millbank, Pimlico, S...",0.00,0.00,0.00,0.025316,0.00,0.012658,0.00,0.00,0.00,0.00,0.00,0.012658,0.00,0.00,0.012658,0.00,0.012658,0.00,0.00,0.00,0.025316,0.000000,0.00,0.00,0.00,0.025316,0.025316,0.012658,0.00,0.012658,0.00,0.000000,0.012658,0.012658,0.00,0.00,0.012658,0.012658,0.00,0.00,0.00,0.012658,0.00,0.050633,0.00,0.012658,0.000000,0.00,0.00,0.00,0.00,0.012658,0.00,0.012658,0.0

In [92]:
westminster_grouped.shape

(7, 127)

each neighborhood along with the top 5 most common venues

In [93]:
num_top_venues = 5

for hood in westminster_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = westminster_grouped[westminster_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aldwych, Charing Cross, Covent Garden----
            venue  freq
0             Pub  0.12
1     Coffee Shop  0.10
2  Sandwich Place  0.09
3      Restaurant  0.07
4           Hotel  0.05


----Bayswater, Paddington----
                venue  freq
0                 Pub  0.08
1                Café  0.07
2  Persian Restaurant  0.04
3               Hotel  0.04
4         Pizza Place  0.04


----Belgravia, Knightsbridge, Millbank, Pimlico, St James's, Westminster----
               venue  freq
0             Lounge  0.06
1              Hotel  0.06
2  French Restaurant  0.05
3             Palace  0.04
4           Tea Room  0.04


----Chinatown, Marylebone (also St Marylebone), Mayfair, Soho----
                venue  freq
0             Theater  0.05
1               Plaza  0.04
2               Hotel  0.04
3              Bakery  0.03
4  Chinese Restaurant  0.03


----Lisson Grove, St John's Wood----
                  venue  freq
0                  Café  0.31
1       Thai Restaurant  0.08
2   

In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

the top 10 venues for each neighborhood

In [95]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = westminster_grouped['Neighbourhood']

for ind in np.arange(westminster_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(westminster_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Aldwych, Charing Cross, Covent Garden",Pub,Coffee Shop,Sandwich Place,Restaurant,Hotel,French Restaurant,Bar,History Museum,Japanese Restaurant,Grocery Store
1,"Bayswater, Paddington",Pub,Café,Persian Restaurant,Hotel,Pizza Place,Indian Restaurant,Coffee Shop,Gym / Fitness Center,Greek Restaurant,Garden
2,"Belgravia, Knightsbridge, Millbank, Pimlico, S...",Lounge,Hotel,French Restaurant,Palace,Tea Room,Seafood Restaurant,Men's Store,Park,Cocktail Bar,Clothing Store
3,"Chinatown, Marylebone (also St Marylebone), Ma...",Theater,Plaza,Hotel,Bakery,Chinese Restaurant,Sandwich Place,Cocktail Bar,Garden,Japanese Restaurant,Ice Cream Shop
4,"Lisson Grove, St John's Wood",Café,Thai Restaurant,Pizza Place,Caucasian Restaurant,Garden,Middle Eastern Restaurant,Yoga Studio,Pub,Deli / Bodega,Persian Restaurant


Cluster Neighborhoods

In [96]:
# set number of clusters
kclusters = 5

westminster_grouped_clustering = westminster_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(westminster_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 4, 1, 0, 2, 1], dtype=int32)

In [97]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

westminster_merged = westminster_data

# merge westminster_grouped with westminster_data to add latitude/longitude for each neighbourhood
westminster_merged = westminster_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

westminster_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Westminster,"Lisson Grove, St John's Wood",51.537125,-0.181958,0,Café,Thai Restaurant,Pizza Place,Caucasian Restaurant,Garden,Middle Eastern Restaurant,Yoga Studio,Pub,Deli / Bodega,Persian Restaurant
1,Westminster,"Belgravia, Knightsbridge, Millbank, Pimlico, S...",51.504456,-0.140785,4,Lounge,Hotel,French Restaurant,Palace,Tea Room,Seafood Restaurant,Men's Store,Park,Cocktail Bar,Clothing Store
2,Westminster,"Chinatown, Marylebone (also St Marylebone), Ma...",51.507408,-0.127699,1,Theater,Plaza,Hotel,Bakery,Chinese Restaurant,Sandwich Place,Cocktail Bar,Garden,Japanese Restaurant,Ice Cream Shop
3,Westminster,"Bayswater, Paddington",51.515390,-0.192100,2,Pub,Café,Persian Restaurant,Hotel,Pizza Place,Indian Restaurant,Coffee Shop,Gym / Fitness Center,Greek Restaurant,Garden
4,Westminster,Maida Vale,51.507408,-0.127699,1,Theater,Plaza,Hotel,Bakery,Chinese Restaurant,Sandwich Place,Cocktail Bar,Garden,Japanese Restaurant,Ice Cream Shop


In [98]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(westminster_merged['Latitude'], westminster_merged['Longitude'], westminster_merged['Neighbourhood'], westminster_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
Based on the defining categories, we can then assign a name to each cluster

Cluster 1

In [99]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 0, westminster_merged.columns[[1] + list(range(5, westminster_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Lisson Grove, St John's Wood",Café,Thai Restaurant,Pizza Place,Caucasian Restaurant,Garden,Middle Eastern Restaurant,Yoga Studio,Pub,Deli / Bodega,Persian Restaurant


Cluster 2

In [100]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 1, westminster_merged.columns[[1] + list(range(5, westminster_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Chinatown, Marylebone (also St Marylebone), Ma...",Theater,Plaza,Hotel,Bakery,Chinese Restaurant,Sandwich Place,Cocktail Bar,Garden,Japanese Restaurant,Ice Cream Shop
4,Maida Vale,Theater,Plaza,Hotel,Bakery,Chinese Restaurant,Sandwich Place,Cocktail Bar,Garden,Japanese Restaurant,Ice Cream Shop


Cluster 3

In [101]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 2, westminster_merged.columns[[1] + list(range(5, westminster_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Bayswater, Paddington",Pub,Café,Persian Restaurant,Hotel,Pizza Place,Indian Restaurant,Coffee Shop,Gym / Fitness Center,Greek Restaurant,Garden
5,Little Venice,Pub,Café,Persian Restaurant,Hotel,Pizza Place,Indian Restaurant,Coffee Shop,Gym / Fitness Center,Greek Restaurant,Garden


Cluster 4

In [102]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 3, westminster_merged.columns[[1] + list(range(5, westminster_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Aldwych, Charing Cross, Covent Garden",Pub,Coffee Shop,Sandwich Place,Restaurant,Hotel,French Restaurant,Bar,History Museum,Japanese Restaurant,Grocery Store


Cluster 5

In [103]:
westminster_merged.loc[westminster_merged['Cluster Labels'] == 4, westminster_merged.columns[[1] + list(range(5, westminster_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Belgravia, Knightsbridge, Millbank, Pimlico, S...",Lounge,Hotel,French Restaurant,Palace,Tea Room,Seafood Restaurant,Men's Store,Park,Cocktail Bar,Clothing Store
